In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
digits = pd.read_csv('./data/digit_recognizer_train.csv')

In [5]:
X = digits.iloc[:, 1:]
y = digits.iloc[:, 0]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [8]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
input_nodes = X_train.shape[1]
hid1_nodes = 500
hid2_nodes = 500
hid3_nodes = 500
hid4_nodes = 500
output_nodes = 10

In [10]:
tf_x = tf.placeholder(name="tf_x", shape=(None, input_nodes), dtype=tf.float32)
tf_y = tf.placeholder(name="tf_y", shape=(None), dtype=tf.int32)

In [11]:
with tf.variable_scope("network", reuse=tf.AUTO_REUSE):
    hid_layer1 = tf.layers.dense(name="hid_layer1", inputs=tf_x, units=hid1_nodes)
    bn_hid_layer1 = tf.layers.batch_normalization(inputs=hid_layer1, training=False, momentum=0.9)
    bn_hid_layer1_act = tf.nn.relu(bn_hid_layer1)

    hid_layer2 = tf.layers.dense(name="hid_layer2", inputs=bn_hid_layer1_act, units=hid2_nodes)
    bn_hid_layer2 = tf.layers.batch_normalization(inputs=hid_layer2, training=False, momentum=0.9)
    bn_hid_layer2_act = tf.nn.relu(bn_hid_layer2)

    hid_layer3 = tf.layers.dense(name="hid_layer3", inputs=bn_hid_layer2_act, units=hid3_nodes)
    bn_hid_layer3 = tf.layers.batch_normalization(inputs=hid_layer3, training=False, momentum=0.9)
    bn_hid_layer3_act = tf.nn.relu(bn_hid_layer3)

    hid_layer4 = tf.layers.dense(name="hid_layer4", inputs=bn_hid_layer3_act, units=hid4_nodes)
    bn_hid_layer4 = tf.layers.batch_normalization(inputs=hid_layer4, training=False, momentum=0.9)
    bn_hid_layer4_act = tf.nn.relu(bn_hid_layer4)

    out_layer = tf.layers.dense(name="out_layer", inputs=bn_hid_layer4_act, units=output_nodes)
    bn_out_layer = tf.layers.batch_normalization(inputs=out_layer, training=False, momentum=0.9)

In [12]:
with tf.variable_scope("cost", reuse=tf.AUTO_REUSE):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(name="loss", labels=tf_y, logits=bn_out_layer)
    cost = tf.reduce_mean(loss, name="cost")

In [13]:
with tf.variable_scope("optimize", reuse=tf.AUTO_REUSE):
    optimizer = tf.train.GradientDescentOptimizer(name="optimizer", learning_rate=0.01)
    optimizer_op = optimizer.minimize(cost)
    
    acc, acc_op = tf.metrics.accuracy(labels=tf_y, predictions=tf.argmax(input=bn_out_layer, axis=1), name="accuracy")

In [14]:
a = np.arange(10)
df = pd.DataFrame(a)
print(list(df.index))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [15]:
def shuffle_batch(X, y, batch_size):
    n_batches = len(X) // batch_size
    X_batches = np.array_split(X, n_batches)
    y_batches = np.array_split(y, n_batches)
    
    for batch_idx in range(n_batches):
        X_batch, y_batch = X_batches[batch_idx], y_batches[batch_idx]
        yield X_batch, y_batch

In [ ]:
n_epochs = 200
batch_size = 64

init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

with tf.Session() as sess:
    init_g.run()
    init_l.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(optimizer_op, feed_dict={tf_x: X_batch, tf_y: y_batch})
        if( (epoch+1) % 20 == 0):
            print("Epoch : ", epoch + 1 ,"    - Train Accuracy : ", \
                      sess.run(acc_op, feed_dict={tf_x:X_train, tf_y:y_train}) \
                     ,"  Test - ", sess.run(acc_op, feed_dict={tf_x:X_test, tf_y:y_test}))
            
            print("*"*50)
    print("Done")        

Epoch :  20     - Train Accuracy :  0.999375   Test -  0.9920476
**************************************************
Epoch :  40     - Train Accuracy :  0.9955688   Test -  0.99235713
**************************************************
Epoch :  60     - Train Accuracy :  0.9945408   Test -  0.99250793
**************************************************
Epoch :  80     - Train Accuracy :  0.9940852   Test -  0.9925774
**************************************************
Epoch :  100     - Train Accuracy :  0.99381447   Test -  0.99261427
**************************************************
Epoch :  120     - Train Accuracy :  0.99363303   Test -  0.9926429
**************************************************
Epoch :  140     - Train Accuracy :  0.9935084   Test -  0.99265987
**************************************************
